<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run using with **Python 3.7** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.7 in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning 
  * DB2

  
The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness, drift monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

# Setup <a name="setup"></a>

## Package installation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --upgrade pyspark==2.4 --no-cache | tail -n 1

!pip install --upgrade pandas==0.25.3 --no-cache | tail -n 1
!pip install --upgrade requests==2.23 --no-cache | tail -n 1
!pip install numpy==1.16.4 --no-cache | tail -n 1
!pip install SciPy --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

ERROR: sparkmagic 0.15.0 requires nose, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires jupyter>=1, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires nose, which is not installed.
ERROR: sparkmagic 0.15.0 requires nose, which is not installed.
ERROR: sparkmagic 0.15.0 requires nose, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires jupyter>=1, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires nose, which is not installed.
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.


### Action: restart the kernel!

## Configure credentials

- WOS_CREDENTIALS (CP4D)
- WML_CREDENTIALS (CP4D)
- DATABASE_CREDENTIALS (DB2 on CP4D or Cloud Object Storage (COS))
- SCHEMA_NAME

In [3]:
WOS_CREDENTIALS = {
    "url": "******",
    "username": "******",
    "password": "******"
}

In [4]:
WML_CREDENTIALS = {
                   "url": "https://******",
                   "username": "******",
                   "password" : "******",
                   "instance_id": "wml_local",
                   "version" : "3.5"
                  }

In [5]:
#IBM DB2 database connection format example
DATABASE_CREDENTIALS = {
    "hostname":"******",
    "username":"******",
    "password":"******",
    "database":"******",
    "port":"******",
    "ssl":False
    #"sslmode":"verify-full",
    #"certificate_base64":"***"
}

### Action: put created schema name below.

In [6]:
SCHEMA_NAME = 'AIOSFASTPATHICP'

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Spark MLLib model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [7]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

rm: cannot remove 'german_credit_data_biased_training.csv': No such file or directory
--2021-03-11 22:53:21--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.02s   

2021-03-11 22:53:22 (26.9 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [8]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## Explore data

In [9]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [10]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


## Visualize data with pixiedust

In [11]:
#import pixiedust

In [12]:
display(df_data)

DataFrame[CheckingStatus: string, LoanDuration: int, CreditHistory: string, LoanPurpose: string, LoanAmount: int, ExistingSavings: string, EmploymentDuration: string, InstallmentPercent: int, Sex: string, OthersOnLoan: string, CurrentResidenceDuration: int, OwnsProperty: string, Age: int, InstallmentPlans: string, Housing: string, ExistingCreditsCount: int, Job: string, Dependents: int, Telephone: string, ForeignWorker: string, Risk: string]

## Create a model

In [13]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 4016
Number of records for evaluation: 984
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = 

The code below creates a Random Forest Classifier with Spark, setting up string indexers for the categorical features and the label column. Finally, this notebook creates a pipeline including the indexers and the model, and does an initial Area Under ROC evaluation of the model.

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")

In [15]:
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [16]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)

**Note**: If you want filter features from model output please replace `*` with feature names to be retained in `SQLTransformer` statement.

In [17]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderROC')
area_under_curve = evaluatorDT.evaluate(predictions)

evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderPR')
area_under_PR = evaluatorDT.evaluate(predictions)
#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve, "areaUnderPR = %g" % area_under_PR)

areaUnderROC = 0.700557 areaUnderPR = 0.609099


In [18]:
# extra code: evaluate more metrics by exporting them into pandas and numpy
from sklearn.metrics import classification_report
y_pred = predictions.toPandas()['prediction']
y_pred = ['Risk' if pred == 1.0 else 'No Risk' for pred in y_pred]
y_test = test_data.toPandas()['Risk']
print(classification_report(y_test, y_pred, target_names=['Risk', 'No Risk']))

              precision    recall  f1-score   support

        Risk       0.79      0.89      0.83       657
     No Risk       0.70      0.51      0.59       327

    accuracy                           0.76       984
   macro avg       0.74      0.70      0.71       984
weighted avg       0.76      0.76      0.75       984



## Save training data to Cloud Object Storage

## Cloud object storage details
In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.
COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [19]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [20]:
COS_API_KEY_ID = "******"
COS_RESOURCE_CRN = "crn:******::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "https://******" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [21]:
BUCKET_NAME = "******" #example: "credit-risk-training-data"

In [22]:
training_data_file_name="german_credit_data_biased_training.csv"

In [23]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [24]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Publish the model

In this section, the notebook uses Watson Machine Learning to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [25]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.44'

In [26]:
space_name = "tutorial-space"
# create the space and set it as default
space_meta_data = {
        wml_client.spaces.ConfigurationMetaNames.NAME : space_name,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION : 'tutorial_space'
}
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
if space_id is None:
    space_id = wml_client.spaces.store(meta_props=space_meta_data)["metadata"]["id"]

print(space_id)
wml_client.set.default_space(space_id)

2bb31cb0-728a-4145-a520-67bc64eafe92


'SUCCESS'

In [27]:
spaces = wml_client.spaces.get_details()['resources']

In [28]:
spaces

[{'entity': {'compute': [{'crn': 'crn:v1:cpd:private:pm-20:private:a/cpduser:99999999-9999-9999-9999-999999999999::',
     'guid': '99999999-9999-9999-9999-999999999999',
     'name': 'Watson Machine Learning',
     'type': 'machine_learning'}],
   'description': '',
   'name': 'scottda-deployment-space',
   'scope': {'bss_account_id': 'cpdaccount'},
   'status': {'state': 'active'}},
  'metadata': {'created_at': '2020-12-08T20:52:39.775Z',
   'creator_id': '1000331001',
   'id': '2569df16-56fc-4a20-bdd6-433dcbb47c77',
   'updated_at': '2020-12-08T20:52:44.899Z',
   'url': '/v2/spaces/2569df16-56fc-4a20-bdd6-433dcbb47c77'}},
 {'entity': {'compute': [{'crn': 'crn:v1:cpd:private:pm-20:private:a/cpduser:99999999-9999-9999-9999-999999999999::',
     'guid': '99999999-9999-9999-9999-999999999999',
     'name': 'Watson Machine Learning',
     'type': 'machine_learning'}],
   'description': 'Deployments for Health Care assets.',
   'name': 'sda-deployment-space',
   'scope': {'bss_account_id'

### Remove existing model and deployment

In [29]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id 2822ac6e-977f-473f-8af5-5ca24e6d3b7b
Deleting model id d8139ae6-1706-43fb-a482-49692946b2b3
--  ----  -------  ----
ID  NAME  CREATED  TYPE
--  ----  -------  ----


#### Add training data reference either from DB2 on CP4D or Cloud Object Storage

In [30]:
# COS training data reference example format

training_data_references = [
                {
                    "id": "Credit Risk",
                    "type": "s3",
                    "connection": {
                        "access_key_id": COS_API_KEY_ID,
                        "endpoint_url": COS_ENDPOINT,
                        "resource_instance_id":COS_RESOURCE_CRN
                    },
                    "location": {
                        "bucket": BUCKET_NAME,
                        "path": training_data_file_name,
                    }
                }
            ]

In [31]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_2.4")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        #wml_client._models.ConfigurationMetaNames.SPACE_UID: space_id,
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_2.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
    }

Software Specification ID: 390d21f8-e58b-4fac-9c55-d7ceda621326


In [32]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_data=train_data, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: a74868c4-be4b-40ec-9efb-05250a23cd4b


In [33]:
wml_client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  ---------
ID                                    NAME                             CREATED                   TYPE
a74868c4-be4b-40ec-9efb-05250a23cd4b  Spark German Risk Model - Final  2021-03-11T22:54:25.002Z  mllib_2.4
------------------------------------  -------------------------------  ------------------------  ---------


## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [34]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: 'a74868c4-be4b-40ec-9efb-05250a23cd4b' started

#######################################################################################


initializing....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2a62b7c6-6951-41bd-9a97-9f0e186a0568'
------------------------------------------------------------------------------------------------


Scoring URL:https://zen-cpd-zen.jrtorres-cpd3-os45-v2-2bef1f4b4097001da9502000c44fc2b2-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/2a62b7c6-6951-41bd-9a97-9f0e186a0568/predictions
Model id: a74868c4-be4b-40ec-9efb-05250a23cd4b
Deployment id: 2a62b7c6-6951-41bd-9a97-9f0e186a0568


## Sample scoring

In [35]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [36]:
scoring_response = wml_client.deployments.score(deployment_uid, scoring_payload)
scoring_response

{'predictions': [{'fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'Job',
    'Dependents',
    'Telephone',
    'ForeignWorker',
    'CheckingStatus_IX',
    'CreditHistory_IX',
    'LoanPurpose_IX',
    'ExistingSavings_IX',
    'EmploymentDuration_IX',
    'Sex_IX',
    'OthersOnLoan_IX',
    'OwnsProperty_IX',
    'InstallmentPlans_IX',
    'Housing_IX',
    'Job_IX',
    'Telephone_IX',
    'ForeignWorker_IX',
    'label',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [['no_checking',
     13,
     'credits_paid_to_date',
     'car_new',
     1343,
     '100_to_500',
     '1_to_4',
     2,
     'female',
     'none',
     3,


# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [37]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

In [38]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],service_instance_id='00000000-0000-0000-0000-1614796138825286', authenticator=authenticator)
wos_client.version

'3.0.1'

## Create datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [39]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-1614796138825286,Data Mart created by OpenScale ExpressPath,False,active,2021-03-03 18:41:43.808000+00:00,00000000-0000-0000-0000-1614796138825286


In [40]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.DB2,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DATABASE_CREDENTIALS['hostname'],
                        username=DATABASE_CREDENTIALS['username'],
                        password=DATABASE_CREDENTIALS['password'],
                        db=DATABASE_CREDENTIALS['database'],
                        port=DATABASE_CREDENTIALS['port'],
                        ssl=DATABASE_CREDENTIALS['ssl'],
                        sslmode=DATABASE_CREDENTIALS['sslmode'],
                        certificate_base64=DATABASE_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 00000000-0000-0000-0000-1614796138825286


## Remove existing service provider connected with used WML instance.
Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [41]:
SERVICE_PROVIDER_NAME = "Watson Machine Learning V2"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [42]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: e0d540b6-d3dc-445e-86fa-87bf743337b7


## Add service provider
Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [43]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(
            url=WML_CREDENTIALS["url"],
            username=WML_CREDENTIALS["username"],
            password=WML_CREDENTIALS["password"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider fa969205-80b1-4ed1-82d5-ea715b785411 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [44]:
wos_client.service_providers.show()

99999999-9999-9999-9999-999999999999,active,Watson Machine Learning V2,watson_machine_learning,2021-03-11 22:55:17.379000+00:00,fa969205-80b1-4ed1-82d5-ea715b785411
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2021-03-03 18:42:06.659000+00:00,2b189084-6cbf-402f-bff9-e138c80582bc
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML production binding,watson_machine_learning,2021-03-03 18:42:00.851000+00:00,5f824736-86ce-48b9-9419-6a91aa5d1f8e


In [45]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id = space_id).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': '2a62b7c6-6951-41bd-9a97-9f0e186a0568',
  'created_at': '2021-03-11T22:54:37.279Z',
  'modified_at': '2021-03-11T22:54:37.279Z'},
 'entity': {'name': 'Spark German Risk Deployment - Final',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/deployments/2a62b7c6-6951-41bd-9a97-9f0e186a0568/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [46]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=space_id)
model_asset_details_from_deployment

{'metadata': {'guid': '2a62b7c6-6951-41bd-9a97-9f0e186a0568',
  'created_at': '2021-03-11T22:54:37.279Z',
  'modified_at': '2021-03-11T22:54:37.279Z'},
 'entity': {'name': 'Spark German Risk Deployment - Final',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/deployments/2a62b7c6-6951-41bd-9a97-9f0e186a0568/predictions'},
  'asset': {'asset_id': 'a74868c4-be4b-40ec-9efb-05250a23cd4b',
   'url': 'https://ibm-nginx-svc.zen.svc.cluster.local/ml/v4/models/a74868c4-be4b-40ec-9efb-05250a23cd4b?space_id=2bb31cb0-728a-4145-a520-67bc64eafe92&version=2020-06-12',
   'name': 'Spark German Risk Model - Final',
   'asset_type': 'model',
   'created_at': '2021-03-11T22:54:25.120Z',
   'modified_at': '2021-03-11T22:54:32.007Z'},
  'asset_properties': {'model_type': 'mllib_2.4',
   'runtime_environment': 'spark-2.4',
   'label_column': 'Risk',
   'input_data_schema': {'type': 'struct',
    'id': '1',
    'fields': [{'name': 'CheckingStatus',
      '

## Subscriptions

### Remove existing credit risk subscriptions

This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [47]:
wos_client.subscriptions.show()

ebb1a9e0-3514-42c2-bf35-8bf5c1031378,GermanCreditRiskModelICP,00000000-0000-0000-0000-1614796138825286,85aed096-8595-4723-a21c-2b5b17f50fb0,GermanCreditRiskModelICP,5f824736-86ce-48b9-9419-6a91aa5d1f8e,active,2021-03-03 18:46:57.339000+00:00,bcc692f8-7ef4-4afd-8511-e445adbbc772
aa527349-fc7c-4d91-94ff-ba144137e3a6,GermanCreditRiskModelPreProdICP,00000000-0000-0000-0000-1614796138825286,5295cbd6-4fc6-4421-8281-b2fb488fb8ea,GermanCreditRiskModelPreProdICP,2b189084-6cbf-402f-bff9-e138c80582bc,active,2021-03-03 18:45:29.541000+00:00,3cc74421-17d2-4690-ac36-c32184320d4d
70e71357-cc60-4708-9fe1-43a638bcd5f8,GermanCreditRiskModelChallengerICP,00000000-0000-0000-0000-1614796138825286,50598d64-2a9c-495f-b8d8-7012c7b28d0f,GermanCreditRiskModelChallengerICP,2b189084-6cbf-402f-bff9-e138c80582bc,active,2021-03-03 18:42:34.833000+00:00,ca70048b-b5ac-4bb2-9053-e81aea2a4290


In [48]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [49]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['probability'],
            prediction_field='predictedLabel',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=TrainingDataReference(type='cos',
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": IAM_URL})),
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

'06964dc3-db53-4535-ba53-189926f458b5'

In [52]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  6fcd8ec3-e6ff-4d83-acf2-7b1d48f19bdd


In [53]:
wos_client.data_sets.show()

00000000-0000-0000-0000-1614796138825286,active,06964dc3-db53-4535-ba53-189926f458b5,subscription,manual_labeling,2021-03-11 22:55:40.511000+00:00,cfd4af6c-6e3d-40b2-b513-c1eaa90ee87d
00000000-0000-0000-0000-1614796138825286,active,06964dc3-db53-4535-ba53-189926f458b5,subscription,payload_logging,2021-03-11 22:55:40.330000+00:00,6fcd8ec3-e6ff-4d83-acf2-7b1d48f19bdd
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,manual_labeling,2021-03-03 18:46:59.977000+00:00,70aa062d-d85d-4d02-a331-af2cbf4b2fb4
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,manual_labeling,2021-03-03 18:45:31.253000+00:00,ec1a0e12-0573-4167-9133-266089e87440
00000000-0000-0000-0000-1614796138825286,active,ca70048b-b5ac-4bb2-9053-e81aea2a4290,subscription,manual_labeling,2021-03-03 18:42:44.681000+00:00,1718662a-4be4-4cde-88e1-ee48464b8383
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,payload_logging,2021-03-03 18:46:59.850000+00:00,b4a3eaa2-445c-4f04-ab97-92ada27dfb0d
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,feedback,2021-03-03 18:47:36.892000+00:00,4e4f020d-7256-45ed-8d0e-fadd8eda5792
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,payload_logging,2021-03-03 18:45:31.152000+00:00,91434557-3edb-46df-a623-b0b2f42a55bb
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,training,2021-03-03 18:47:00.067000+00:00,5cd17974-558c-4355-987c-a64d34671d70
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,feedback,2021-03-03 18:45:59.413000+00:00,9c35ef98-8b40-46b9-9bb6-26bf6680c69b


Note: First 10 records were displayed.


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model.

In [54]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.

## Check if WML payload logging worked else manually store payload records

In [55]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
#time.sleep(5)
wml_predictions = {"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']}
#print(wml_predictions)
#print("==========")
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response=wml_predictions,
                   response_time=460
               )])
    time.sleep(15)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 0
Payload logging did not happen, performing explicit payload logging.
Number of records in the payload logging table: 8


In [56]:
time.sleep(10)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 8



# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [57]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 56b5a05c-e0cc-49d8-8d2f-cff80edcfec2 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [58]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'56b5a05c-e0cc-49d8-8d2f-cff80edcfec2'

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [59]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/additional_feedback_data_v2.json

rm: cannot remove 'additional_feedback_data_v2.json': No such file or directory
--2021-03-12 16:45:19--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.003s  

2021-03-12 16:45:20 (17.4 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



## Get feedback logging dataset ID

In [60]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "9265f4a4-90c5-4ee2-96ef-6e15670a8803",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-1614796138825286:data_set:9265f4a4-90c5-4ee2-96ef-6e15670a8803",
        "url": "/v2/data_sets/9265f4a4-90c5-4ee2-96ef-6e15670a8803",
        "created_at": "2021-03-12T16:45:13.714000Z",
        "created_by": "internal-service",
        "modified_at": "2021-03-12T16:45:14.408000Z",
        "modified_by": "internal-service"
      },
      "entity": {
        "data_mart_id": "00000000-0000-0000-0000-1614796138825286",
        "name": "06964dc3-db53-4535-ba53-189926f458b5_feedback",
        "description": "06964dc3-db53-4535-ba53-189926f458b5_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "06964dc3-db53-4535-ba53-189926f458b5"
        },
        "schema_update_mode": "auto",
        "data_schema": {
          "type": "struct",
   

In [61]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [62]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: f4191f21-4de5-4ce2-9a72-f119b4e91ee0 




pending
active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [63]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [64]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 7ffc0185-6dd3-4ea4-b2d4-592e86d80282 




running
error

-------------------------------
 Run failed with status: error 
-------------------------------


Reason: ['code: AIQFS0002E, message: Action `Score Batch` has failed with status code 500; associated message: `{2}`', "code: AIQGS0099E, message: User '1000330999' is not authorized to perform this action on space '2bb31cb0-728a-4145-a520-67bc64eafe92'."]


In [65]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# Fairness, drift monitoring and explanations 
 <a name="fairness"></a>

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [66]:
wos_client.monitor_instances.show()

00000000-0000-0000-0000-1614796138825286,active,06964dc3-db53-4535-ba53-189926f458b5,subscription,quality,2021-03-12 16:45:12.796000+00:00,56b5a05c-e0cc-49d8-8d2f-cff80edcfec2
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,drift,2021-03-03 18:47:42.095000+00:00,5c4b4626-7a3c-4f83-9e3d-d6a2eb447bef
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,fairness,2021-03-03 18:47:23.117000+00:00,d9d5f0b1-641b-400a-a16d-64338609567d
00000000-0000-0000-0000-1614796138825286,active,b4a3eaa2-445c-4f04-ab97-92ada27dfb0d,instance,performance,2021-03-03 18:50:38.059000+00:00,c5249a88-9c98-4115-adf6-7d7aee80aef1
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,fairness,2021-03-03 18:45:52.727000+00:00,5994153e-05af-40ff-9881-96f74981daaa
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,mrm,2021-03-03 18:47:48.164000+00:00,7e20767a-c1b5-4a49-9dd3-62cadbb39387
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,quality,2021-03-03 18:47:34.931000+00:00,e04329ee-3f50-40a8-8ae7-7a4814ecd1de
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,drift,2021-03-03 18:46:05.792000+00:00,e6ab3892-9ddf-4e7b-a580-0ef883f9aeb5
00000000-0000-0000-0000-1614796138825286,active,bcc692f8-7ef4-4afd-8511-e445adbbc772,subscription,explainability,2021-03-03 18:47:15.330000+00:00,5194f768-7bcb-4bf3-87d7-d6acdf5a08c4
00000000-0000-0000-0000-1614796138825286,active,3cc74421-17d2-4690-ac36-c32184320d4d,subscription,mrm,2021-03-03 18:46:11.603000+00:00,238abb72-35b1-4d0e-8c77-723ac675b50a


Note: First 10 records were displayed.


In [67]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 56176a69-4eb9-4da5-ad93-0bbc186664f8 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'56176a69-4eb9-4da5-ad93-0bbc186664f8'

## Drift configuration

In [68]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [69]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 100,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation c796359b-e9bb-4c1f-81ce-066c88355b84 




preparing.
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'c796359b-e9bb-4c1f-81ce-066c88355b84'

## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [70]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_feed.json
!ls -lh german_credit_feed.json

--2021-03-12 16:46:09--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076279 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json’

german_credit_feed. 100%[===================>]   2.93M  --.-KB/s    in 0.06s   

2021-03-12 16:46:10 (48.7 MB/s) - ‘german_credit_feed.json’ saved [3076279/3076279]

-rw-r-----. 1 wsuser watsonstudio 3.0M Mar 12 16:46 german_credit_feed.json


Score 200 randomly chosen records

In [71]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)
time.sleep(5)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response=scoring_response,
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Payload logging did not happen, performing explicit payload logging.
Number of records in the payload logging table: 8


In [72]:
print('Number of records in payload table: ', wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table:  8


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [105]:
time.sleep(5)
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 7b1e9880-8e5d-45ea-8012-1d7b0023a788 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [106]:
time.sleep(10)
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2021-03-12 17:54:19.309629+00:00,fairness_value,9b20b361-ccf1-4316-ac00-425c384aa30a,0.947,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:minority']",fairness,56176a69-4eb9-4da5-ad93-0bbc186664f8,7b1e9880-8e5d-45ea-8012-1d7b0023a788,subscription,06964dc3-db53-4535-ba53-189926f458b5
2021-03-12 17:54:19.309629+00:00,fairness_value,9b20b361-ccf1-4316-ac00-425c384aa30a,0.954,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:minority']",fairness,56176a69-4eb9-4da5-ad93-0bbc186664f8,7b1e9880-8e5d-45ea-8012-1d7b0023a788,subscription,06964dc3-db53-4535-ba53-189926f458b5
2021-03-12 17:45:53.195529+00:00,fairness_value,b7eddd3b-392b-4871-bc81-ccf0e5efcbf4,0.947,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:minority']",fairness,56176a69-4eb9-4da5-ad93-0bbc186664f8,194c2051-ea69-4d70-81a2-de8d7ae7ea67,subscription,06964dc3-db53-4535-ba53-189926f458b5
2021-03-12 17:45:53.195529+00:00,fairness_value,b7eddd3b-392b-4871-bc81-ccf0e5efcbf4,0.954,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:minority']",fairness,56176a69-4eb9-4da5-ad93-0bbc186664f8,194c2051-ea69-4d70-81a2-de8d7ae7ea67,subscription,06964dc3-db53-4535-ba53-189926f458b5


## Run drift monitor

Kick off a drift monitor run on current data. The monitor runs every hour, but can be manually initiated using the Python client, the REST API.

In [75]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 1dff14a7-1730-4787-a660-7ac0bc044b5e 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [76]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2021-03-12 16:46:47.143379+00:00,data_drift_magnitude,abc304ad-e1ed-44b5-963c-086429258563,0.08173076923076923,None,None,[],drift,c796359b-e9bb-4c1f-81ce-066c88355b84,1dff14a7-1730-4787-a660-7ac0bc044b5e,subscription,06964dc3-db53-4535-ba53-189926f458b5


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [77]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation d48c570c-4255-4962-a869-8321c33b87c2 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [78]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)

Running explanations on scoring IDs: ['d3ef547e-20d8-4e03-bb35-2afe5a286c35-1']
{
  "metadata": {
    "explanation_task_ids": [
      "d9f85f0e-cadc-4f41-b7c8-7aa09f549916"
    ],
    "created_by": "1000331001",
    "created_at": "2021-03-12T16:47:08.276782Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [79]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [80]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition




 Waiting for end of adding monitor definition my_model_performance 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------




## Show available monitors types

In [81]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
assurance,Assurance,"['Uncertainty', 'Confidence']"
fairness,Fairness,"['Fairness value', 'Average Odds Difference metric value', 'False Discovery Rate Difference metric value', 'Error Rate Difference metric value', 'False Negative Rate Difference metric value', 'False Omission Rate Difference metric value', 'False Positive Rate Difference metric value', 'True Positive Rate Difference metric value']"
performance,Performance,['Number of records']
explainability,Explainability,[]
mrm,Model risk management monitoring,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
correlations,Correlations,"['Maximum positive correlation coefficient', 'Maximum negative correlation coefficient', 'Mean absolute correlation coefficient', 'Significant correlation coefficients count']"
drift,Drift,"['Drop in accuracy', 'Predicted accuracy', 'Drop in data consistency']"
quality,Quality,"['Area under ROC', 'Area under PR', 'Proportion explained variance', 'Mean absolute error', 'Mean squared error', 'R squared', 'Root of mean squared error', 'Accuracy', 'Weighted True Positive Rate (wTPR)', 'True positive rate (TPR)', 'Weighted False Positive Rate (wFPR)', 'False positive rate (FPR)', 'Weighted recall', 'Recall', 'Weighted precision', 'Precision', 'Weighted F1-Measure', 'F1-Measure', 'Logarithmic loss']"


### Get monitors uids and details

In [82]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [83]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-1614796138825286:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2021-03-12T16:47:11.603000Z",
    "created_by": "scottda"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region",
        "description": "customer

## Enable custom monitor for subscription

In [84]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target
).result




 Waiting for end of monitor instance creation a6c20e0f-0493-487b-98cc-e9fbba3905fe 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [85]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [86]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "a6c20e0f-0493-487b-98cc-e9fbba3905fe",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-1614796138825286:monitor_instance:a6c20e0f-0493-487b-98cc-e9fbba3905fe",
    "url": "/v2/monitor_instances/a6c20e0f-0493-487b-98cc-e9fbba3905fe",
    "created_at": "2021-03-12T16:47:20.324000Z",
    "created_by": "scottda"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-1614796138825286",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "06964dc3-db53-4535-ba53-189926f458b5"
    },
    "thresholds": [
      {
        "metric_id": "sensitivity",
        "type": "lower_limit",
        "value": 0.8
      },
      {
        "metric_id": "specificity",
        "type": "lower_limit",
        "value": 0.75
      }
    ],
    "schedule": {
      "repeat_interval": 60,
      "repeat_unit": "minute",
      "repeat_type": "minute"
    },
    "status": {
   

## Storing custom metrics

In [87]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = "11122223333111abc"
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
print(measurement_request[0])

{
  "timestamp": "2021-03-12T16:47:25.707010Z",
  "run_id": "11122223333111abc",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [88]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': '49d57dcc-0301-4c4b-bedd-4db3a172147b', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'run_id': '11122223333111abc', 'timestamp': '2021-03-12T16:47:25.707010Z'}]


### List and get custom metrics

In [89]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "49d57dcc-0301-4c4b-bedd-4db3a172147b",
    "crn": "",
    "url": "/v2/monitor_instances/a6c20e0f-0493-487b-98cc-e9fbba3905fe/measurements/49d57dcc-0301-4c4b-bedd-4db3a172147b",
    "created_at": "2021-03-12T16:47:26.170000Z",
    "created_by": ""
  },
  "entity": {
    "timestamp": "2021-03-12T16:47:25.707010Z",
    "run_id": "11122223333111abc",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.8
          },
          {
            "id": "specificity",
            "value": 0.78,
            "lower_limit": 0.75
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "target": {
      "target_type": "subscription",
      "target_id": "06964dc3-db53-4535-ba53-189926f458b5"
    },
    "monitor_instance_id": "a6c20e0f-0493-487b-98cc-e9fbba3905fe",


# Historical data <a name="historical"></a>

In [90]:
historyDays = 7

 ## Insert historical payloads

The next section of the notebook downloads and writes historical data to the payload and measurement tables to simulate a production model that has been monitored and receiving regular traffic for the last seven days. This historical data can be viewed in the Watson OpenScale user interface. The code uses the Python and REST APIs to write this data.

In [91]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
!ls -lh history_fairness_v2.json

--2021-03-12 16:47:32--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37259 (36K) [text/plain]
Saving to: ‘history_fairness_v2.json’

history_fairness_v2 100%[===================>]  36.39K  --.-KB/s    in 0.002s  

2021-03-12 16:47:32 (16.7 MB/s) - ‘history_fairness_v2.json’ saved [37259/37259]

-rw-r-----. 1 wsuser watsonstudio 37K Mar 12 16:47 history_fairness_v2.json


In [92]:
from datetime import datetime, timedelta, timezone

with open('history_fairness_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
 
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        daily_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical debias metrics

In [93]:
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
!ls -lh history_debias_v2.json

--2021-03-12 16:47:41--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37259 (36K) [text/plain]
Saving to: ‘history_debias_v2.json’

history_debias_v2.j 100%[===================>]  36.39K  --.-KB/s    in 0.002s  

2021-03-12 16:47:42 (16.6 MB/s) - ‘history_debias_v2.json’ saved [37259/37259]

-rw-r-----. 1 wsuser watsonstudio 37K Mar 12 16:47 history_debias_v2.json


In [94]:
with open('history_debias_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        
        daily_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical quality metrics

In [95]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical confusion matrixes

In [96]:
!rm history_quality_metrics.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
!ls -lh history_quality_metrics.json

rm: cannot remove 'history_quality_metrics.json': No such file or directory
--2021-03-12 16:48:01--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80099 (78K) [text/plain]
Saving to: ‘history_quality_metrics.json’

history_quality_met 100%[===================>]  78.22K  --.-KB/s    in 0.004s  

2021-03-12 16:48:01 (19.1 MB/s) - ‘history_quality_metrics.json’ saved [80099/80099]

-rw-r-----. 1 wsuser watsonstudio 79K Mar 12 16:48 history_quality_metrics.json


In [97]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical performance metrics

In [98]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )


performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id




 Waiting for end of monitor instance creation 54235f35-004a-42f6-8451-ac1abd9fdcfd 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [99]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical drift measurements

In [101]:
!rm history_drift_measurement_*.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_3.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_4.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_5.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

rm: cannot remove 'history_drift_measurement_*.json': No such file or directory
--2021-03-12 16:53:26--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 850981 (831K) [text/plain]
Saving to: ‘history_drift_measurement_0.json’

history_drift_measu 100%[===================>] 831.04K  --.-KB/s    in 0.03s   

2021-03-12 16:53:27 (28.6 MB/s) - ‘history_drift_measurement_0.json’ saved [850981/850981]

--2021-03-12 16:53:28--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
Resolving raw.githubuserconte

In [102]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1
Daily loading finished.
Loading day 2
Daily loading finished.
Loading day 3
Daily loading finished.
Loading day 4
Daily loading finished.
Loading day 5
Daily loading finished.
Loading day 6
Daily loading finished.
Loading day 7
Daily loading finished.


## Additional data to help debugging

In [103]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)

Datamart: 00000000-0000-0000-0000-1614796138825286
Model: a74868c4-be4b-40ec-9efb-05250a23cd4b
Deployment: 2a62b7c6-6951-41bd-9a97-9f0e186a0568


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [104]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

None,5054,d3ef547e-20d8-4e03-bb35-2afe5a286c35-1,0.0,3.0,1.0,3,less_100,2021-03-12T16:46:17.049Z,0.7228899574974899,None,"[20, [0, 2, 4, 6, 7, 13, 14, 15, 16, 17, 18, 19], [1.0, 3.0, 3.0, 1.0, 1.0, 22.0, 5054.0, 3.0, 2.0, 35.0, 1.0, 1.0]]",less_0,radio_tv,0.0,No Risk,1.0,0.0,skilled,none,0.0,0.0,co-applicant,22,35,prior_payments_delayed,0.0,2,0.0,3.0,0.0,0.0,yes,own,0.0,1,2a62b7c6-6951-41bd-9a97-9f0e186a0568,"[14.457799149949798, 5.542200850050202]",none,1.0,male,1,car_other,less_1,"[0.7228899574974899, 0.2771100425025101]"
None,4780,d3ef547e-20d8-4e03-bb35-2afe5a286c35-10,0.0,3.0,1.0,2,500_to_1000,2021-03-12T16:46:17.049Z,0.8120494991505649,None,"[1.0, 1.0, 3.0, 2.0, 3.0, 0.0, 0.0, 0.0, 2.0, 1.0, 3.0, 0.0, 0.0, 30.0, 4780.0, 2.0, 3.0, 29.0, 1.0, 1.0]",less_0,radio_tv,0.0,No Risk,0.0,0.0,unemployed,bank,2.0,1.0,none,30,29,credits_paid_to_date,2.0,3,0.0,3.0,3.0,0.0,yes,rent,1.0,1,2a62b7c6-6951-41bd-9a97-9f0e186a0568,"[16.240989983011296, 3.7590100169887015]",none,0.0,male,1,savings_insurance,less_1,"[0.8120494991505649, 0.1879505008494351]"
None,2229,d3ef547e-20d8-4e03-bb35-2afe5a286c35-100,1.0,0.0,1.0,1,less_100,2021-03-12T16:46:17.049Z,0.9251734935763165,None,"[20, [0, 1, 4, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 4.0, 4.0, 1.0, 14.0, 2229.0, 1.0, 2.0, 23.0, 1.0, 1.0]]",less_0,car_new,0.0,No Risk,0.0,0.0,skilled,none,0.0,4.0,none,14,23,no_credits,0.0,2,0.0,4.0,0.0,0.0,yes,own,0.0,1,2a62b7c6-6951-41bd-9a97-9f0e186a0568,"[18.503469871526327, 1.4965301284736696]",none,0.0,female,1,savings_insurance,unemployed,"[0.9251734935763165, 0.07482650642368349]"
None,250,d3ef547e-20d8-4e03-bb35-2afe5a286c35-101,1.0,0.0,2.0,2,less_100,2021-03-12T16:46:17.049Z,0.9316862446757551,None,"[2.0, 4.0, 0.0, 0.0, 3.0, 1.0, 0.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 12.0, 250.0, 2.0, 1.0, 20.0, 1.0, 1.0]",0_to_200,car_new,0.0,No Risk,2.0,0.0,unskilled,bank,0.0,4.0,none,12,20,no_credits,2.0,1,0.0,3.0,1.0,0.0,yes,rent,1.0,1,2a62b7c6-6951-41bd-9a97-9f0e186a0568,"[18.633724893515105, 1.366275106484898]",none,0.0,female,1,real_estate,less_1,"[0.9316862446757551, 0.06831375532424488]"
None,413,d3ef547e-20d8-4e03-bb35-2afe5a286c35-102,1.0,0.0,2.0,2,less_100,2021-03-12T16:46:17.049Z,0.8561738244407818,None,"[20, [0, 4, 5, 8, 13, 14, 15, 16, 17, 18, 19], [2.0, 1.0, 1.0, 1.0, 6.0, 413.0, 2.0, 2.0, 29.0, 1.0, 1.0]]",0_to_200,car_new,0.0,No Risk,0.0,0.0,skilled,stores,0.0,0.0,none,6,29,prior_payments_delayed,1.0,2,0.0,1.0,0.0,0.0,yes,own,0.0,1,2a62b7c6-6951-41bd-9a97-9f0e186a0568,"[17.123476488815637, 2.876523511184364]",none,0.0,female,1,savings_insurance,4_to_7,"[0.8561738244407818, 0.1438261755592182]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the OpenScale Dashboard: (https://url-to-your-cp4d-cluster/aiopenscale). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.
